In [1]:
import sys, os
sys.path.append("C:/Users/chlgy/Documents/deep-learning-from-scratch/ch03")
from dataset.mnist import load_mnist

(x_train,t_train),(x_test,t_test)= load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [2]:
import numpy as np
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train,t_train), (x_test,t_test)=load_mnist(flatten=True, normalize=False)

img=x_train[0]
label=t_train[0]
print(label)

print(img.shape)
img=img.reshape(28,28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


이 신경망은 입력층 뉴런을 784개, 출력층 뉴런을 10개로 구성한다. 입력층 뉴런이 784개인 이유는 이미지 크기가 28 * 28 =784이기 때문이다.  출력층 뉴런이 10개인 이유는 이 문제가 0에서 9까지의 숫자를 구분하는 문제이기 때문이다.

In [5]:
import pickle

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def softmax(a):
    exp_a=np.exp(a)
    sum_exp_a=np.sum(exp_a)
    y= exp_a/sum_exp_a
    
    return y


def get_data():
    (x_train,t_train),(x_test,t_test)=load_mnist(normalize=True, flatten=True, one_hot_label=False)
    
    return x_test,t_test

def init_network():
    with open("sample_weight.pkl",'rb') as f:
        network=pickle.load(f)
        
        return network
    
def predict(network,x):
    W1,W2,W3=network['W1'],network['W2'],network['W3']
    b1,b2,b3=network['b1'],network['b2'],network['b3']
    
    a1=np.dot(x,W1)+b1
    z1=sigmoid(a1)
    a2=np.dot(z1,W2)+b2
    z2=sigmoid(a2)
    a3=np.dot(z2,W3)+b3
    z3=softmax(a3)
    
    return y

In [8]:
x,t=get_data()
network=init_network()

accuracy_cnt=0
for i in range(len(x)):
    y= predict(network,x[i])
    p=np.argmax(y)
    if p == t[i]:
        accuracy_cnt +=1
        
print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

Accuracy:0.9352


In [7]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax


def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352


정확도 0.9352는 올바르게 분류한 비율이 93.52%라는 뜻이다. 

In [9]:
print(y)

[4.2882870e-04 2.0043037e-06 2.5405674e-03 2.0168918e-06 5.5917754e-04
 3.1262074e-04 9.9614763e-01 4.3499412e-07 6.3756934e-06 3.7751448e-07]


In [10]:
print(p)

6


predict() 함수는 각 레이블의 확률을 넘파이 배열로 반환한다. 그런 다음 np.argmax() 함수로 이 배열에서 가장 값이 큰 원소의 인덱스를 구한다.  

신경망 입력데이터에 특정 변환을 가하는 것을 **전처리**라 한다. 여기에서는 전처리 작업으로 정규화를 수행한 셈이다.

In [11]:
x, _ = get_data()
network=init_network()
W1,W2,W3 = network['W1'],network['W2'],network['W3']

x.shape

(10000, 784)

In [12]:
x[0].shape

(784,)

In [13]:
W1.shape

(784, 50)

In [14]:
W2.shape

(50, 100)

In [15]:
W3.shape

(100, 10)

In [17]:
# 배치 처리 구현

batch_size=100
accuracy_cnt=0

for i in range(0,len(x), batch_size):
    x_batch=x[i:i+batch_size]
    y_batch=predict(network,x_batch)
    p=np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p==t[i:i+batch_size])

print("Accuracy:" +str(float(accuracy_cnt)/len(x)))

Accuracy:0.9352


range(start, end, step)처럼 인수를 3개 지정하면 start에서 end-1까지 step 간격으로 증가하는 리스트를 반환한다.

In [18]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [19]:
list(range(0,10,3))

[0, 3, 6, 9]

x[i:i+batch_size]는 입력데이터의 i번째 부터 i+batch_size번째 까지의 데이터를 묶는다는 것  

batch_size=100이라는 것은 x[0:100], x[100:200]...을 의미한다.

In [20]:
x=np.array([[0.1,0.8,0.1],[0.3,0.1,0.6],[0.2,0.5,0.3],[0.8,0.1,0.1]])

y=np.argmax(x, axis=1) # 100*10 배열 중에서 1차원을 구성하는 각 원소에서 최댓값의 인덱스를 찾도록 하는 axis
print(y)

[1 2 1 0]


In [21]:
y=np.array([1,2,1,0])
t=np.array([1,2,0,0])
print(y==t)

[ True  True False  True]


In [22]:
np.sum(y==t) # ==을 이용해 bool 배열을 만들고 True가 몇개 인지 센다

3